In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

In [2]:
collisions = pd.read_csv("h2017collisionsfinal.csv")

Get first 50 rows for demo

In [3]:
collisions = collisions.loc[:50]

In [4]:
collisions.head()

,Record,Location,X,Y,Longitude,Latitude,Year,Date,Time,Environment,Road_Surface,Traffic_Control,Collision_Location,Light,Collision_Classification,Impact_type
0,1,210 W OF MERIVALE RD @ WEST HUNT CLUB RD,365431.70335,5.021829e+06,-75.72644,45.33379,2017,2017-08-22,6:10:00 PM,01 - Clear,02 - Wet,01 - Traffic signal,02 - Intersection related,01 - Daylight,02 - Non-fatal injury,03 - Rear end
1,2,210 W OF MERIVALE RD @ WEST HUNT CLUB RD,365430.63705,5.021830e+06,-75.72645,45.33380,2017,2017-10-19,12:20:00 PM,01 - Clear,01 - Dry,01 - Traffic signal,03 - At intersection,01 - Daylight,03 - P.D. only,05 - Turning movement
2,3,210 W OF MERIVALE RD @ WEST HUNT CLUB RD,365431.70110,5.021829e+06,-75.72644,45.33379,2017,2017-01-31,8:09:00 AM,01 - Clear,01 - Dry,01 - Traffic signal,02 - Intersection related,01 - Daylight,02 - Non-fatal injury,03 - Rear end
3,4,210 W OF MERIVALE RD @ WEST HUNT CLUB RD,365431.35217,5.021829e+06,-75.72644,45.33379,2017,2017-03-24,10:45:00 AM,03 - Snow,03 - Loose snow,01 - Traffic signal,02 - Intersection related,01 - Daylight,03 - P.D. only,03 - Rear end
4,5,210 W OF MERIVALE RD @ WEST HUNT CLUB RD,365432.02107,5.021829e+06,-75.72643,45.33379,2017,2017-04-04,12:30:00 PM,02 - Rain,02 - Wet,01 - Traffic signal,02 - Intersection related,01 - Daylight,03 - P.D. only,03 - Rear end


Create the dimensions for location, accident and weather

In [5]:
Accident_dimension = pd.DataFrame(columns=['Accident_surogate_key','Accident_ID', 'Environment', 'Road_Surface', 'Traffic_Control',
                                     'Visibility', 'Impact_Type'])

Location_dimension = pd.DataFrame(columns=['Location_ID','Street_Name', 'Intersection_1', 'Intersection_2', 'Longitude', 'Latitude'])




In [6]:
for idx, row in collisions.iterrows():
    
    #Accident
    accident_row = [idx + 100]
    accident_row.append(row["Record"])
    accident_row.append(row['Environment'][4:].strip()) 
    accident_row.append(row['Road_Surface'][4:].strip()) 
    accident_row.append(row['Traffic_Control'][4:].strip()) 
    accident_row.append(row['Light'][4:].strip())  
    accident_row.append(row['Impact_type'][4:].strip())
    Accident_dimension.loc[len(Accident_dimension)] = accident_row
    
    
    #Location
    location_row = [idx]
    if '@' in row['Location']: 
        roads = row['Location'].split("@")
        location_row += [roads[0].strip(), roads[1].strip(), "NA"] #Street, Intersection 1, Intersection 2
        
    else: #when there is a btwn in the location
        roads = row['Location'].split("btwn")
        
        if "&" in roads[1]:
            intersections = roads[1].split("&")
            location_row += [roads[0].strip(), intersections[0].strip(), 
                             intersections[1].strip()] #Street, Intersection 1, Intersection 2
        else:
            location_row += [roads[0].strip(), roads[1].strip(), "NA"] #Street, Intersection 1, Intersection 2
        
    location_row += [row['Longitude'], row['Latitude']]
    
    Location_dimension.loc[len(Location_dimension)] = location_row
    
    


In [7]:
Accident_dimension.to_csv("accident_dimension.csv",index=False)

In [8]:
Accident_dimension.head()

,Accident_surogate_key,Accident_ID,Environment,Road_Surface,Traffic_Control,Visibility,Impact_Type
0,100,1,Clear,Wet,Traffic signal,Daylight,Rear end
1,101,2,Clear,Dry,Traffic signal,Daylight,Turning movement
2,102,3,Clear,Dry,Traffic signal,Daylight,Rear end
3,103,4,Snow,Loose snow,Traffic signal,Daylight,Rear end
4,104,5,Rain,Wet,Traffic signal,Daylight,Rear end


In [9]:
Location_dimension = Location_dimension.drop_duplicates(subset=['Longitude','Latitude'])
Location_dimension.insert(0, "surrogate_key",  np.arange(len(Location_dimension)))

In [10]:
Location_dimension.head()

,surrogate_key,Location_ID,Street_Name,Intersection_1,Intersection_2,Longitude,Latitude
0,0,0,210 W OF MERIVALE RD,WEST HUNT CLUB RD,NA,-75.72644,45.33379
1,1,1,210 W OF MERIVALE RD,WEST HUNT CLUB RD,NA,-75.72645,45.33380
4,2,4,210 W OF MERIVALE RD,WEST HUNT CLUB RD,NA,-75.72643,45.33379
6,3,6,225 E OF RIVERSIDE DR / T & T SC,HUNT CLUB RD,NA,-75.69064,45.33558
7,4,7,2ND LINE RD,CABIN RD,NA,-75.59877,45.16056


In [11]:
Location_dimension.to_csv("location_dimension.csv",index=False)

Pre-process weather

In [12]:
climate = pd.read_csv("climate-daily.csv")

/Users/yi/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (33,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
climate[climate["STATION_NAME"].str.match("OTTAWA")].head()

,x,y,STATION_NAME,CLIMATE_IDENTIFIER,ID,LOCAL_DATE,PROVINCE_CODE,LOCAL_YEAR,LOCAL_MONTH,LOCAL_DAY,...,SPEED_MAX_GUST,SPEED_MAX_GUST_FLAG,COOLING_DEGREE_DAYS,COOLING_DEGREE_DAYS_FLAG,HEATING_DEGREE_DAYS,HEATING_DEGREE_DAYS_FLAG,MIN_REL_HUMIDITY,MIN_REL_HUMIDITY_FLAG,MAX_REL_HUMIDITY,MAX_REL_HUMIDITY_FLAG
28483,-75.716667,45.383333,OTTAWA CDA,6105976,6105976.2016.8.1,2016-08-01 00:00:00,ON,2016,8,1,...,NaN,NaN,5.8,NaN,0.0,NaN,NaN,NaN,NaN,NaN
28484,-75.716667,45.383333,OTTAWA CDA,6105976,6105976.2016.8.2,2016-08-02 00:00:00,ON,2016,8,2,...,NaN,NaN,4.3,NaN,0.0,NaN,NaN,NaN,NaN,NaN
28485,-75.716667,45.383333,OTTAWA CDA,6105976,6105976.2016.8.3,2016-08-03 00:00:00,ON,2016,8,3,...,NaN,NaN,4.5,NaN,0.0,NaN,NaN,NaN,NaN,NaN
28486,-75.716667,45.383333,OTTAWA CDA,6105976,6105976.2016.8.4,2016-08-04 00:00:00,ON,2016,8,4,...,NaN,NaN,7.8,NaN,0.0,NaN,NaN,NaN,NaN,NaN
28487,-75.716667,45.383333,OTTAWA CDA,6105976,6105976.2016.8.5,2016-08-05 00:00:00,ON,2016,8,5,...,NaN,NaN,8.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN


Easy solution - Choosing one station (OTTAWA INTL A)

In [14]:
ottawa_climate = climate[climate["STATION_NAME"] == ("OTTAWA INTL A")]

Deleting the columns with most missing values

In [15]:
ottawa_climate = ottawa_climate[['x', 'y', 'STATION_NAME', 'CLIMATE_IDENTIFIER', 'ID', 'LOCAL_DATE',
       'PROVINCE_CODE', 'LOCAL_YEAR', 'LOCAL_MONTH', 'LOCAL_DAY','MEAN_TEMPERATURE',  'MIN_TEMPERATURE',
       'MAX_TEMPERATURE', 
       'TOTAL_PRECIPITATION',  'TOTAL_RAIN',
        'TOTAL_SNOW', 'DIRECTION_MAX_GUST', 
       'SPEED_MAX_GUST', 'COOLING_DEGREE_DAYS',
      'HEATING_DEGREE_DAYS']]

Adding the sorrogate key to the weather dimension.

In [16]:
ottawa_climate.insert(0, "surrogate_key",  np.arange(len(ottawa_climate)))

In [17]:
ottawa_climate["LOCAL_DATE"] = pd.to_datetime(ottawa_climate["LOCAL_DATE"] ).dt.strftime('%Y-%m-%d')

In [18]:
ottawa_climate['ID'] = ottawa_climate['ID'].str.replace('.','')

In [19]:
ottawa_climate = ottawa_climate.dropna()

In [20]:
weather_dimension = ottawa_climate

In [21]:
weather_dimension.to_csv("weather_dimension.csv",index=False)

In [22]:
weather_dimension.head()

,surrogate_key,x,y,STATION_NAME,CLIMATE_IDENTIFIER,ID,LOCAL_DATE,PROVINCE_CODE,LOCAL_YEAR,LOCAL_MONTH,...,MEAN_TEMPERATURE,MIN_TEMPERATURE,MAX_TEMPERATURE,TOTAL_PRECIPITATION,TOTAL_RAIN,TOTAL_SNOW,DIRECTION_MAX_GUST,SPEED_MAX_GUST,COOLING_DEGREE_DAYS,HEATING_DEGREE_DAYS
29512,0,-75.666667,45.316667,OTTAWA INTL A,6106001,6106001201681,2016-08-01,ON,2016,8,...,23.6,16.8,30.3,0.0,0.0,0.0,8.0,41.0,5.6,0.0
29513,1,-75.666667,45.316667,OTTAWA INTL A,6106001,6106001201682,2016-08-02,ON,2016,8,...,22.6,14.9,30.3,0.0,0.0,0.0,7.0,35.0,4.6,0.0
29514,2,-75.666667,45.316667,OTTAWA INTL A,6106001,6106001201683,2016-08-03,ON,2016,8,...,22.8,14.0,31.6,0.0,0.0,0.0,20.0,33.0,4.8,0.0
29515,3,-75.666667,45.316667,OTTAWA INTL A,6106001,6106001201684,2016-08-04,ON,2016,8,...,26.2,18.2,34.2,0.0,0.0,0.0,24.0,33.0,8.2,0.0
29516,4,-75.666667,45.316667,OTTAWA INTL A,6106001,6106001201685,2016-08-05,ON,2016,8,...,25.8,18.5,33.1,0.0,0.0,0.0,18.0,50.0,7.8,0.0
